In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data

dataX=pd.read_csv("./data/trainX.csv")
dataA=pd.read_csv("./data/trainA.csv")
dataY=pd.read_csv("./data/trainY.csv")

# remove index

dataX=dataX.values[:,1:]
dataY=dataY.values[:,1:-1]
dataA=dataA.values[:,1:]

In [3]:
# split cure or not cure
치료한환자데이터=pd.read_csv("./data/2번 열을 제거한 치료한 환자 데이터.csv")
치료하지않은환자데이터=pd.read_csv("./data/8번 열을 제거한 치료하지 않은 환자 데이터.csv")

In [4]:
data_CY=치료한환자데이터['time']
dataNCY=치료하지않은환자데이터['time']
data_CX = 치료한환자데이터.loc[:,['X0','X1','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16']]
dataNCX = 치료하지않은환자데이터.loc[:,['X0','X1','X2','X3','X4','X5','X6','X7','X9','X10','X11','X12','X13','X14','X15','X16']]

In [5]:
from keras import models
from keras import layers

Using TensorFlow backend.
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\kosta\anaconda3\envs\ge

In [9]:
import tensorflow as tf

In [30]:
def build_dnn_relu_adam():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[16])
  
  H = tf.keras.layers.Dense(64)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 1
  
  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 2
  
  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 3

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 4

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 5

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 6

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 7
    
  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 8

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 9

  H = tf.keras.layers.Dense(64)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 10

  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])
  return model

def build_model_NC_100_stepper():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[16])
  
  H = tf.keras.layers.Dense(2048)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  H = tf.keras.layers.Dense(1024)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)

  H = tf.keras.layers.Dense(256)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)

  H = tf.keras.layers.Dense(128)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])
  return model

In [31]:
# 치료 데이터
num_epochs = 1000
all_mae_histories = []
k = 4
num_val_samples = len(data_CX) // k
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = data_CX[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = data_CY[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [data_CX[:i * num_val_samples],
         data_CX[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [data_CY[:i * num_val_samples],
         data_CY[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_dnn_relu_adam()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['mean_absolute_error']
    all_mae_histories.append(mae_history)

처리중인 폴드 # 0


KeyError: 'val_mean_absolute_error'

In [ ]:
# 치료하지 않은 데이터
num_epochs = 2000
all_mae_histories_non_cured = []
k = 4
num_val_samples = len(dataNCX) // k
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = dataNCX[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = dataNCY[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [dataNCX[:i * num_val_samples],
         dataNCX[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [dataNCY[:i * num_val_samples],
         dataNCY[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_dnn_swish_adam()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history_non_cured = history.history['mean_absolute_error']
    all_mae_histories_non_cured.append(mae_history)

In [ ]:
# 3.데이터로 모델을 학습(FIT)합니다.
model = build_dnn_relu_adam() # 최저 loss 값 : 0.0492
model.fit(data_CX, data_CY, epochs=20000, verbose=0)
model.fit(data_CX, data_CY, epochs=1, verbose=1)

In [ ]:
# 3.치료하지 않았고 생존시간이 100 이하인 환자 데이터로 모델을 학습(FIT)합니다.
model_NC = build_dnn_relu_adam() # 최저 loss 값 : 0.0267
model.fit(data_CX, data_CY, epochs=20000, verbose=0)
model.fit(data_CX, data_CY, epochs=1, verbose=1)

In [ ]:
치료_환자_파일경로 = './data/2번 열을 제거한 치료한 환자 테스트 데이터.csv'
비치료_환자_파일경로 = './data/8번 열을 제거한 치료하지 않은 환자 테스트 데이터.csv'
치료_환자_데이터 = pd.read_csv(치료_환자_파일경로)
비치료_환자_데이터 = pd.read_csv(비치료_환자_파일경로)

In [ ]:
# 치료할 경우 생존 시간을 구한다.
cured = model.predict(치료_환자_데이터)
# 치료하지 않을 경우 생존 시간을 구한다.
non_cured = model_NC.predict(비치료_환자_데이터)

# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 1, 
# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 0
result = np.where(cured > non_cured, 1, 0)
print(result.shape)

In [ ]:
import csv 

# 이차원 리스트 
with open('./data/result_20200902_1.csv','w', newline='') as f: 
  makewrite = csv.writer(f) 
  
  for value in result: 
    makewrite.writerow(value)
